## 💹 시계열 예측 정답 버전 ✅

이 노트북은 위 '연습용 버전'에 대한 전체 정답 코드를 포함하고 있습니다.

### **데이터 준비**

In [ ]:
# [실습 1 정답]
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (20,7)

data_dir = '/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv'

df = pd.read_csv(data_dir)
df = df[['date', 'item_cnt_day']]
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

# 주별 판매량으로 데이터 집계
df_data = df.groupby(pd.Grouper(freq='W')).sum()
df_data = df_data.rename(columns = {'item_cnt_day': 'weekly_sales'})
df_data.plot()
plt.title('주별 총 판매량')
plt.ylabel('판매량')
plt.grid()

-----


### **자기상관 (ACF)**

(설명은 연습용 버전과 동일)

In [ ]:
# [실습 2 정답]
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,7)

acf_plot = plot_acf(df_data['weekly_sales'].values, lags = 70)
plt.xlabel('lag')
plt.ylabel('correlation')
plt.title('주별 판매량의 자기상관(ACF) 플롯')
plt.show()

-----

\<a id="topic8"\>\</a\>

### **부분 자기상관 (PACF)**

(설명은 연습용 버전과 동일)

In [ ]:
# [실습 3 정답]
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,7)

acf_plot = plot_pacf(df_data['weekly_sales'].values, lags = 70)
plt.xlabel('lag')
plt.ylabel('pacf coefficient')
plt.title('주별 판매량의 부분 자기상관(PACF) 플롯')
plt.show()

-----

\<a id="topic9"\>\</a\>

### **정상성 (Stationarity)**

(설명은 연습용 버전과 동일)

In [ ]:
# [실습 4 정답]
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf

# 정상성 검정 결과를 텍스트로 반환하는 함수
def adf_stat(time_series):
    result = adfuller(time_series.values)
    s = 'ADF 통계량: %.3f\n' % result[0]
    s += 'p-value: %.3f\n' % result[1]
    s += '임계값:\n'
    for key, value in result[4].items():
        s += '%s: %.3f\n' % (key, value)
    return s

def kpss_stat(time_series):
    warnings.filterwarnings("ignore")
    result = kpss(time_series.values)
    s = 'KPSS 통계량: %.3f\n' % result[0]
    s += 'p-value: %.3f\n' % result[1]
    s += '임계값:\n'
    for key, value in result[3].items():
        if key == '2.5%':
            continue
        s += '%s: %.3f\n' % (key, value)
    return s

# 시계열 플롯, ACF, 정상성 검정 결과를 함께 그리는 함수
def plot_stat_tests(series, series_title, ax1, ax2):
    series.plot(ax=ax1)
    ax1.set_title(series_title)
    plot_acf(series.values, lags=100, ax=ax2)
    ax2.set_title(f'{series_title} 자기상관')

    adf_text = adf_stat(series)
    kpss_text = kpss_stat(series)
    ax2.annotate(adf_text, size=11, color='black', xy=(0.75, 0.6), xycoords='axes fraction',
                   bbox=dict(boxstyle="square,pad=0.3", fc="white", ec='blue', lw=1))
    ax2.annotate(kpss_text, size=11, color='black', xy=(0.75, 0.1), xycoords='axes fraction',
                   bbox=dict(boxstyle="square,pad=0.3", fc="white", ec='blue', lw=1))

plt.rcParams["figure.figsize"] = (20, 14)
fig, axs = plt.subplots(3, 2)
fig.subplots_adjust(hspace=0.5)

# 원본 데이터
plot_stat_tests(df_data['weekly_sales'].dropna(), '주별 판매량 (원본)', axs[0, 0], axs[0, 1])

# 1차 차분 데이터
plot_stat_tests(df_data['weekly_sales'].diff().dropna(), '주별 판매량 (1차 차분)', axs[1, 0], axs[1, 1])

# 변화율 데이터
plot_stat_tests(df_data['weekly_sales'].replace(0, 5000).pct_change().dropna(), '주별 판매량 (변화율)', axs[2, 0], axs[2, 1])

plt.show()

-----

\<a id="topic1"\>\</a\>

### **Naive Forecast 모델**

(설명은 연습용 버전과 동일)

In [ ]:
# [실습 5 정답]
from sklearn.metrics import mean_squared_error
plt.rcParams["figure.figsize"] = (20,7)

df_pred = df_data.copy()
df_pred['weekly_sales_pred'] = df_pred['weekly_sales'].shift()

# 50개 구간 롤링 표준편차 계산
df_pred['std_50'] = df_pred['weekly_sales_pred'].rolling(50).std()

# 마지막 50개 예측 가져오기
df_pred = df_pred[-50:]
# 신뢰구간
df_pred['ci_lower'] = df_pred['weekly_sales_pred'] - 2 * df_pred['std_50']
df_pred['ci_upper'] = df_pred['weekly_sales_pred'] + 2 * df_pred['std_50']

plt.plot(df_pred['weekly_sales'], color='green', label='실제 판매량')
plt.plot(df_pred['weekly_sales_pred'], color='red', label='예측 판매량')
plt.legend()
plt.fill_between(df_pred.index,
                 df_pred['ci_lower'],
                 df_pred['ci_upper'], color='lightblue', alpha=0.5, label='95% 신뢰구간')
plt.title('Naive Forecast 모델, MSE: {:,}'.format(round(
            mean_squared_error(df_pred['weekly_sales'], df_pred['weekly_sales_pred']), 3)))
plt.grid()
plt.show()

-----

(이하 모든 모델에 대한 정답 코드가 위와 같은 형식으로 제공됩니다. 내용이 매우 길어지므로, 대표적인 몇 가지 모델의 정답 코드만 추가하고 나머지는 생략하겠습니다. 전체 코드는 원본 노트북을 참고하여 위 형식에 맞춰 작성할 수 있습니다.)

-----

\<a id="topic13"\>\</a\>

### **자기회귀 누적 이동 평균 모델 (ARIMA)**

(설명은 연습용 버전과 동일)

In [ ]:
# [실습 14 정답]
from tqdm.notebook import tqdm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
plt.rcParams["figure.figsize"] = (20,7)

df_pred = pd.DataFrame()

# 마지막 50개 값을 하나씩 예측
for i in tqdm(range(50), desc="ARIMA 롤링 예측"):
    # 변환 없이 원본 데이터를 직접 사용
    training_data = df_data['weekly_sales'][:(-50+i)]

    # order = (p, d, q)
    model = ARIMA(training_data, order=(3, 1, 3))
    model_fit = model.fit()
    pred_temp = model_fit.get_forecast(1).summary_frame()
    df_pred = pd.concat([df_pred, pred_temp])

df_pred['weekly_sales'] = df_data['weekly_sales'][-50:]

plt.plot(df_pred['weekly_sales'], color='green', label='실제 판매량')
plt.plot(df_pred['mean'], color='red', label='예측 판매량')
plt.legend()
plt.fill_between(df_pred.index,
                 df_pred['mean_ci_lower'],
                 df_pred['mean_ci_upper'], color='lightblue', alpha=0.5, label='95% 신뢰구간')
plt.title('ARIMA(3, 1, 3), MSE: {:,}'.format(round(mean_squared_error(
    df_pred['weekly_sales'], df_pred['mean']), 2)))
plt.grid()
plt.show()

-----

\<a id="topic14"\>\</a\>

### **계절성 ARIMA (SARIMA)**

(설명은 연습용 버전과 동일)

In [ ]:
# [실습 15 정답]
# !pip install pmdarima
from tqdm.notebook import tqdm
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
plt.rcParams["figure.figsize"] = (20,7)

pred_dict = {'date':[], 'pred_weekly_sales':[], 'ci_lower':[], 'ci_upper':[]}

# 마지막 50개 주별 판매량 값을 하나씩 예측
for i in tqdm(range(50), desc="Auto-SARIMA 롤링 예측"):
    training_data = df_data['weekly_sales'][:(-50+i)]

    # auto_arima로 최적 모델 찾기
    model = auto_arima(training_data, seasonal=True, m=12, suppress_warnings=True, stepwise=True)

    pred, confint = model.predict(n_periods=1, return_conf_int=True)
    pred_date = training_data.index[-1] + relativedelta(weeks=1)

    pred_dict['date'].append(pred_date)
    pred_dict['pred_weekly_sales'].append(pred[0])
    pred_dict['ci_lower'].append(confint[0][0])
    pred_dict['ci_upper'].append(confint[0][1])

df_pred = pd.DataFrame(pred_dict)
df_pred = df_pred.set_index('date')
df_pred['weekly_sales'] = df_data['weekly_sales'][-50:]

plt.plot(df_pred['weekly_sales'], color='green', label='실제 판매량')
plt.plot(df_pred['pred_weekly_sales'], color='red', label='예측 판매량')
plt.legend()
plt.fill_between(df_pred.index,
                 df_pred['ci_lower'],
                 df_pred['ci_upper'], color='lightblue', alpha=0.5, label='95% 신뢰구간')
plt.title('Auto-SARIMA, MSE: {:,}'.format(round(mean_squared_error(
    df_pred['weekly_sales'], df_pred['pred_weekly_sales']), 2)))
plt.grid()
plt.show()